### Imports

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Activation, Embedding, LSTM
from sklearn.model_selection import train_test_split
import numpy as np
import pandas 
import plotly as py
import plotly.graph_objs as go
from plotly import tools
from matplotlib.pyplot import figure


### Data Preparation

In [ ]:
data = pandas.read_csv('./data/Learning/last version/Train_Set.csv')
test = pandas.read_csv('./data/Learning/last version/Test_Set.csv')
gtTrain = pandas.read_csv('./data/21/groundTruthEdited.csv')
gtTest = pandas.read_csv('./data/17/groundTruthEdited.csv')

x_train_set = data[['x','y','z']]

target = data['brakingStatus']

x_test_set = test[['x','y','z']]
y_test_set = test['brakingStatus']

In [ ]:
x_train_set = np.array(x_train_set,dtype=float)
target = np.array(target,dtype=float)

x_test_set =  np.array(x_test_set,dtype=float)
y_test_set =  np.array(y_test_set,dtype=float)

In [ ]:
x_train_set_array= []
for i in x_train_set:
    x_train_set_array.append([i])
x_train_set_array = np.array(x_train_set_array, dtype = float)

x_test_set_array= []
for i in x_test_set:
    x_test_set_array.append([i])
x_test_set_array = np.array(x_test_set_array, dtype = float)

y_test_set_array= []
for i in y_test_set:
    y_test_set_array.append([i])
y_test_set_array = np.array(y_test_set_array, dtype = float)

target_array= []      
for i in target:
    target_array.append([i])
target_array = np.array(target_array, dtype = float)




In [ ]:
x_train_set_array_reshape = np.squeeze(x_train_set_array, axis = 1)
target_array_reshape = np.squeeze(target_array, axis = 1)
x_test_set_array_reshape = np.squeeze(x_test_set_array, axis=1)
y_test_set_array_reshape = np.squeeze(y_test_set_array, axis=1)

### one_hot encoding

In [ ]:
# len(x_test_set_array)
# len(x_train_set_array_reshape)
target_array_reshape_onehot  = []
y_test_set_array_reshape_onehot = []
for i in range(len(target_array_reshape)):
    if(target_array_reshape[i]):
        target_array_reshape_onehot.append(np.array([0,1]))
    else:
        target_array_reshape_onehot.append(np.array([1,0]))

        
for i in range(len(y_test_set_array_reshape)):
    if(y_test_set_array_reshape[i]):
        y_test_set_array_reshape_onehot.append(np.array([0,1]))
    else:
        y_test_set_array_reshape_onehot.append(np.array([1,0]))

### RNN model

In [ ]:
model = Sequential()

In [ ]:
model.add(LSTM(1,batch_input_shape= (None,10,3),return_sequences= False))

In [ ]:
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit( 
    np.array([x_train_set_array_reshape[i:i+10] for i in range(0,50600)]),
    np.array([target_array_reshape[i+9] for i in range(0,50600)]),
    epochs=150,
    validation_data=(np.array([x_train_set_array_reshape[i:i+10] for i in range(50601,51050)]),np.array([target_array_reshape[i+9] for i in range(50601,51050)])),
    batch_size=128)
# history = model.fit(
#     x_train_set_array[5000:5010],target_array[5000:5010],epochs=100,
#     validation_data=(x_train_set_array[18500:20000],target_array[18500:20000]),
#     batch_size=128)

In [ ]:
[x_train_set_array[i:i+10] for i in range(1,100)]

In [ ]:
results = model.predict(np.array([x_test_set_array_reshape[i:i+10] for i in range(0,len(x_test_set_array)-10)]))

In [ ]:
# thresholded_result = []
# for i in range(len(results)):
#     if(results[i]<0.5):
#         thresholded_result.append(0)
#     else:
#         thresholded_result.append(1)

# df = pandas.DataFrame(data=thresholded_result,columns=['brakingStatus'])
# df.to_csv('./data/Learning/results.csv',index= False)
results

In [ ]:
thresholded_result = []
for i in range(len(results)):
    if(results[i]<0.5):
        thresholded_result.append(0)
    else:
        thresholded_result.append(1)
        
len(thresholded_result)

### RNN one_hot

In [ ]:
model = Sequential()

In [ ]:
model.add(LSTM(2, batch_input_shape = (None,20,3), return_sequences = True))

In [ ]:
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit( 
    np.array(x_train_set_array_reshape[:50600]).reshape(2530,20,3),
    np.array([target_array_reshape_onehot[i:i+20] for i in range(0,50600,20)]),
    epochs=200,
    validation_data= (np.array(x_train_set_array_reshape[50650:51050]).reshape(20,20,3),
                     np.array([target_array_reshape_onehot[i:i+20] for i in range(50650,51050,20)])),
batch_size=32)

In [ ]:
results = model.predict(np.array(x_test_set_array_reshape[:10660].reshape(533,20,3)))

In [ ]:
results = results.reshape(10660,2)

In [ ]:
result_class = []
for i in results:
    if(np.argmax(i)):
        result_class.append(1)
    else:
        result_class.append(0)
        

In [ ]:
results.shape

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test_set_array_reshape[:10660], result_class, labels=[0,1], average='binary')

In [ ]:

fig =tools.make_subplots(rows=1,cols=1)

result =  go.Scatter(
    x = list(range(0,10677)),
    y = result_class,
    mode = 'lines',
    name='model_result'
)

test =  go.Scatter(
    x = list(range(0,10677)),
    y = y_test_set_array_reshape,
    mode = 'lines',
    name='test'
)

fig.append_trace(result,1,1)
fig.append_trace(test,1,1)


fig['layout'].update(height=500, title = 'Braking Model Classification')


py.offline.plot(fig, filename='braking_onehot_20_sigmoid_binary_crossentropy')


### one_hot visualization

In [ ]:

fig =tools.make_subplots(rows=1,cols=1)

result =  go.Scatter3d(
    x = list(range(0,10677)),
    y = [item[0] for item in results],
    z = [item[1] for item in results],
    name='model_result',
    mode = 'markers'
)

test =  go.Scatter3d(
    x = list(range(0,10677)),
    y = [item[0] for item in y_test_set_array_reshape_onehot],
    z = [item[1] for item in y_test_set_array_reshape_onehot],    
    name='test'
)

fig = go.Figure(data=[result, test])
py.offline.plot(fig, filename='multiple-axes-double')

In [ ]:
fig =tools.make_subplots(rows=2,cols=1)

loss_result =  go.Scatter(
    x = list(range(0,200)),
    y = history.history['loss'],
    mode = 'lines'
)

acc_result =  go.Scatter(
    x = list(range(0,200)),
    y = history.history['acc'],
    mode = 'lines'
)

fig.append_trace(loss_result, 1, 1)
fig.append_trace(acc_result, 2, 1)

fig['layout'].update(height=1000, title = 'Loss, Accuracy')
py.offline.plot(fig, filename='multiple-axes-double')

### Training by just 'X' 

In [ ]:
x_train_set_X = data['x']
x_test_set_X = test['x']


x_train_set_X = np.array(x_train_set_X,dtype= float)
x_test_set_X = np.array(x_test_set_X,dtype= float)


In [ ]:
x_train_set_array_X= []      
for i in x_train_set:
    x_train_set_array_X.append([i])
x_train_set_array_X = np.array(x_train_set_array_X, dtype = float)

x_test_set_array_X= []      
for i in x_test_set:
    x_test_set_array_X.append([i])
x_test_set_array_X = np.array(x_test_set_array_X, dtype = float)

In [ ]:
model_X = Sequential()

In [ ]:
model_X.add(LSTM(1,batch_input_shape= (None,1,3),return_sequences= False))

In [ ]:
model_X.compile(loss='mean_absolute_error',optimizer='adam',metrics=['accuracy'])

In [ ]:
model_X.summary()